#### Define Imports

In [1]:
#%pip install openai

In [2]:
import sys
sys.path.insert(0,'../')
import pandas as pd
from environment import env
from environment import gpt_model_functions
config = env.env()

#### API Call

In [3]:
query = 'Provide the state that Bryce Canyon is in'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
output

'Bryce Canyon National Park is located in Utah'

In [4]:
query = 'Where is Yosemite located?'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
output

'Yosemite National Park is located at 9039 Village Drive in Yosemite, California 95389'

In [8]:
query = 'What alerts are there at Yosemite?'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)


In [9]:
print(output)

There are 4 active alerts for Yosemite National Park: 
 Alert 1: Expect delays at entrance stations from mid-morning to early afternoon. Delays vary by entrance station but can be up to two hours.
 Alert 2: Tree work to reduce fire danger is ongoing in western Yosemite Valley. Expect 15-minute delays, Mondays through Fridays, 6 am to 4 pm.
 Alert 3: A reservation is required to drive into or through Yosemite from 5 am to 4 pm every day from July 1 through August 16, then weekends and holidays only from August 17 through October 27.
 Alert 4: Shuttles are not servicing stops 14–19 (Curry Village, Happy Isles and Mirror Lake trailheads, and Pines campgrounds). A temporary shuttle stop is available near the Curry Village front desk. Access to the closed shuttle stops is by foot or bicycle only.
 


In [9]:
query = 'Provide the description of Bryce Canyon National Park'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
intent

'description'

In [8]:
query = "What's the full name of Bryce Canyon"
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
intent

'fullname'

In [4]:
query = 'How much are the enterance fees for Bryce Canyon National Park?'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
intent

'feespass'

In [5]:
query = 'What are the amenities at Bryce Canyon National Park?'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
intent

'amenities'

In [6]:
query = 'What events are happening at Bryce Canyon National Park?'
endpoint, parkcode, intent, output = gpt_model_functions.api_call(query)
intent

'events'

In [ ]:
model_name = "distilbert-base-uncased-distilled-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a pipeline
nlp = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Load the data
df = pd.read_csv('parks_data.csv')

# Initialize the model and tokenizer


# Function to generate answers
def answer_question(question, context):
    result = nlp(question=question, context=context)
    return result['answer']

# Function to get park context
def get_park_context(park_name):
    park = df[df['name'].str.contains(park_name, case=False, na=False)]
    if not park.empty:
        context = f"{park.iloc[0]['name']} is located at {park.iloc[0]['address']} in {park.iloc[0]['state']}. {park.iloc[0]['description']}"
        return context
    else:
        return "Sorry, I couldn't find the park you're looking for."

# Function to handle user queries
def handle_query(query):
    # Extract the entity (park name) from the query
    entity = extract_entity(query)
    if entity:
        context = get_park_context(entity)
        answer = answer_question(query, context)
        return answer
    else:
        return "Please specify a park in your question."

# Function to extract the entity (park name) from the query
def extract_entity(query):
    for park_name in df['name']:
        if park_name.lower() in query.lower():
            return park_name
    return None

# Example usage
query = "Tell me about Yellowstone National Park"
response = handle_query(query)
print(response)